In [1]:
!pip install 'accelerate>=0.26.0' torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 182.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 156.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 131.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 27.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:

In [2]:
from huggingface_hub import login
import os

# Login
login()

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import logging

logging.basicConfig(level=logging.INFO)

class Agent:
    def __init__(self, name, role, model_name, device=None, temperature=0.7, top_p=0.9, max_tokens=1024):
        self.name = name
        self.role = role  # Claimer, Challenger, or Judge
        self.model_name = model_name
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load tokenizer and model with error handling
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if self.device.type == 'cuda' else torch.float32
            ).to(self.device)
        except Exception as e:
            logging.error(f"Failed to load model {self.model_name}: {e}")
            raise

        # Set up the pipeline with flexible parameters
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=self.device,
            max_new_tokens=1024,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
        )
        self.history = []
        self.score = 1.0

    def generate_response(self, messages):
        """
        Generate a response from the model based on the input messages.
        """
        prompt = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])
        prompt += "\nAssistant:"
        try:
            response = self.pipeline(prompt)[0]['generated_text']
            # Extract the assistant's response
            response = response[len(prompt):].strip().split('\n')[0].strip()
        except Exception as e:
            logging.error(f"Error generating response: {e}")
            return "Sorry, I couldn't generate a response."
        return response


def construct_message(agent, previous_responses, question):
    """
    Construct a message for the agent based on its role.
    """
    if agent.role == 'Claimer':
        # Claimer must fully support the statement
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Claimer. You strongly believe that '{question}' is true. Provide a clear, complete argument supporting this."},
            {'role': 'user', 'content': f"The debate topic is: '{question}'. Present your argument fully and clearly before anyone else responds."}
        ]
    elif agent.role == 'Challenger':
        # Challenger must respond only after the Claimer is done
        responses_summary = "\n".join([f"Claim: {resp}" for resp in previous_responses])
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Challenger. Counter the claim provided by the Claimer using your reasoning skills."},
            {'role': 'user', 'content': f"The claimer made the following argument: {responses_summary}. Provide your rebuttal."}
        ]
    elif agent.role == 'Judge':
        # Judge evaluates both Claimer's and Challenger's responses
        return [
            {'role': 'system', 'content': f"You are {agent.name}, a Judge. Evaluate the arguments provided by both the Claimer and Challenger and decide which argument is stronger."},
            {'role': 'user', 'content': "\n".join(previous_responses)}
        ]


def run_debate(agents, question, rounds=1):
    """
    Run a multi-agent debate where agents respond in a controlled sequential order.
    """
    claimer = next(agent for agent in agents if agent.role == 'Claimer')
    challenger = next(agent for agent in agents if agent.role == 'Challenger')
    judge = next(agent for agent in agents if agent.role == 'Judge')

    # Step 1: Claimer makes an initial claim
    claimer_messages = construct_message(claimer, [], question)
    claimer_response = claimer.generate_response(claimer_messages)
    print(f"{claimer.name} (Claimer): {claimer_response}")

    # Step 2: Challenger waits for the Claimer to finish and then counters the claim
    challenger_messages = construct_message(challenger, [claimer_response], question)
    challenger_response = challenger.generate_response(challenger_messages)
    print(f"{challenger.name} (Challenger): {challenger_response}")

    # Step 3: Continue the debate if there are more rounds
    for round_num in range(2, rounds + 1):
        # Claimer revises or reinforces the claim
        claimer_messages = construct_message(claimer, [challenger_response], question)
        claimer_response = claimer.generate_response(claimer_messages)
        print(f"Round {round_num} - {claimer.name} (Claimer): {claimer_response}")

        # Challenger responds with a counterargument
        challenger_messages = construct_message(challenger, [claimer_response], question)
        challenger_response = challenger.generate_response(challenger_messages)
        print(f"Round {round_num} - {challenger.name} (Challenger): {challenger_response}")

    # Step 4: Judge evaluates the final responses
    judge_messages = construct_message(judge, [claimer_response, challenger_response], question)
    judge_response = judge.generate_response(judge_messages)
    print(f"{judge.name} (Judge): {judge_response}")

    # Return the final result decided by the judge
    return judge_response


if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Create agents with different roles
    agents = [
        Agent(name='Agent1', role='Claimer', model_name='meta-llama/Llama-3.1-8B-Instruct', device=device),
        Agent(name='Agent2', role='Challenger', model_name='Qwen/Qwen2.5-0.5B-Instruct', device=device),
        Agent(name='Agent3', role='Judge', model_name='Qwen/Qwen2.5-0.5B-Instruct', device=device),
    ]

    # Define the debate question
    question = "Zoos should be banned. Do you agree or disagree?"

    # Run the multi-agent debate
    result = run_debate(agents, question)
    print(f"Final decision: {result}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Agent1 (Claimer): As Agent1, I claim that there are valid arguments in support of banning zoos, but I also acknowledge that there are opposing viewpoints. I propose that we evaluate the claim "Zoos should be banned" based on the following points:
Agent2 (Challenger): 1) Zoos are institutions dedicated to preserving and protecting endangered species


KeyboardInterrupt: 